## Modelo Preditivo: Random Forest Regressor


O Random Forest Regressor é um algoritmo de aprendizado de máquina baseado em árvores de decisão, usado para tarefas de regressão (previsão de valores numéricos). Ele funciona criando várias árvores de decisão independentes (como uma "floresta") a partir de subconjuntos diferentes dos dados original. Cada árvore faz uma previsão, e temos como resultado final é a média das previsões de todas as árvores.

### Instruções para uso local ou remoto (Google Colab ou VScode)

Aqui deixaremos brevemente um passo a passo para que você usuário seja capaz de executar o código localmente ou remotamente pelo seu google drive, podendo escolher a forma mais viável para seu uso e conhecimento.

##### Google Colab

1. Faça o upload do seu arquivo .ipynb para o Google Drive.
2. Abra o Google Colab em seu navegador.
3. Clique em "Arquivo" no menu superior e selecione "Abrir notebook".
4. Na guia "Upload", clique em "Procurar" e selecione o arquivo .ipynb que você enviou para o Google Drive.
5. Após selecionar o arquivo, clique em "Abrir".
6. Aguarde o carregamento do notebook no Google Colab.
7. Agora que você carregou o notebook no Google Colab, você pode fazer as alterações necessárias nos arquivos e caminhos para se adequar ao seu ambiente específico.

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("/content/drive/MyDrive/NomeDaPastaDosArquivos/arquivo.csv)

Salve as alterações no notebook.
Agora você pode executar as células do notebook no Google Colab e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você não precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente, dado que ao executar pela ferramenta do google essas dependências estarão aplicadas por padrão

##### VScode

1. Coloque os arquivos CSV dentro da pasta "data" desse notebook

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv")

Salve as alterações no notebook.
Agora você pode executar as células do notebook no VScode e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente:

Para fazer a instalação, basta abrir o terminal integrado e inserir o seguinte:

In [1]:
#pip install pandas numpy matplotlib scikit-learn shap

### Execução do Modelo

O primeiro passo para realizarmos nossas previsões, é importar as bibliotecas que serão usadas.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import shap

Abaixo, usei os arquivos na pasta "data" com os nomes descritos, mas lembre-se de mudar conforme necessecidade;

In [3]:
# # pegando arquivos csv do drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Lendo os arquivos CSV
# tabela_Meta = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/BASE INTELI_META-limpo.csv")
# tabela_Agosto = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

# para realizar o processo localmente, descomente as linhas abaixo e comente as linhas acima.
tabela_Meta = pd.read_csv("../data/dados_tratados/BASE INTELI_META-limpo.csv")
tabela_Agosto = pd.read_csv("../data/dados_tratados/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

In [ ]:
tabela_Agosto.head()

Com os arquivos em mão, vamos definir valores máximos no formato de exibição e agrupar colunas, visando somar as de valor númerico de acordo com as exigências do parceiro. No fim, a saída mostrará uma representação da tabela filtrada com as features usadas será mostrada.

In [ ]:
# Definindo formato de exibição
# pd.set_option('display.float_format', '{:.0f}'.format)
ipca_padrao = tabela_Agosto.groupby(['Ano', 'Mês'])['IPCA ES'].first().reset_index()


# Agrupar e somar 'Vl Liquido Final' e 'Outra Coluna'
tabela_agosto_segmento = tabela_Agosto.groupby(['Ano', 'Mês','Veiculo', 'Origem','Segmento']).agg({
    'Vl Liquido Final': 'sum',
    # 'IPCA ES': 'sum',
}).reset_index()

tabela_agosto_segmento = tabela_agosto_segmento.merge(ipca_padrao, on=['Ano', 'Mês'], how='left')


tabela_agosto_segmento.head()

Portanto, de acordo com a tabela filtrada, as features utilizadas são as:
- **Ano**: Permite avaliar o desempenho da receita ao longo do ano.
- **Mês**: Indica em qual mês a receita teve maior impacto.
- **Vl. Líquido**: Utilizado para prever a receita final, pois os ganhos estão relacionados a esse valor.
- **Vl. Tabela**: Reflete o valor inicial dos contratos, essencial para calcular a diferença entre o valor acordado e o final.
- **Veículo**: Analisa o impacto de cada veículo de venda na receita final.
- **Origem**: Identifica a localidade da venda para verificar a região de maior impacto na receita.
- **Segmento**: Permite filtrar receitas por setor econômico de forma mais detalhada (segmento).

Daqui em diante, iremos trabalhar mais com ajustes das colunas para que o modelo seja capaz de compreender as informarções.

 Abaixo, as colunas categóricas (aquelas que contém texto) serão transformadas em numéricas, de modo a pouco comprometer o desempenho do modelo.

In [6]:
categorical_columns = ['Origem', 'Veiculo', 'Segmento']
X = tabela_agosto_segmento.drop('Vl Liquido Final', axis=1)
y = tabela_agosto_segmento['Vl Liquido Final']

# Aplicando OneHotEncoder nas colunas categóricas
onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)

# Transformar as colunas categóricas e converter para DataFrame
X_encoded = onehot_encoder.fit_transform(X[categorical_columns])

# Converter para DataFrame com os nomes das colunas correspondentes
X_encoded_df = pd.DataFrame(X_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))

# Remover as colunas categóricas originais de X
X = X.drop(categorical_columns, axis=1)

# Concatenar as colunas codificadas com o restante de X
X = pd.concat([X.reset_index(drop=True), X_encoded_df.reset_index(drop=True)], axis=1)

Após, faremos a divisão das amostras de treinamento e teste do modelo, e depois o ajuste do modelo com o grupo de treino, equivalente à 30% da base total

In [7]:
y_log = np.log(y)

In [ ]:
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=42)

modelo = RandomForestRegressor()
modelo.fit(X_train, y_train_log)

Por fim, faremos o modelo realizar previsões com a porcentagem de 70% da base que foi destinada a esse teste.

Podemos ver na saída desse bloco de código as métricas usadas para validar a eficiência do modelo, explicaremos elas em breve.

In [ ]:
# Fazer previsões
y_pred_log = modelo.predict(X_test)

y_pred_exp = np.exp(y_pred_log)
y_test_exp = np.exp(y_test_log)

mae = mean_absolute_error(y_test_exp, y_pred_exp)
print(f"Mean Absolute Error: {mae}")
rmse = mean_squared_error(y_test_exp, y_pred_exp)
print(f"Root Mean Squared Error: {sqrt(rmse)}")
r2 = r2_score(y_test_log, y_pred_log)
print(f"R² Score: {r2*100:.2f}%")
mape = mean_absolute_percentage_error(y_test_exp, y_pred_exp)
print (f"MAPE: {mape*100:.2f}%")


### Erro Médio Absoluto (MAE - do inglês *Mean Absoluto Error*)

O MAE é uma métrica utilizada para avaliar a qualidade de modelos de regressão, calculando a média das diferenças absolutas entre os valores reais e os valores preditos. Quanto menor o valor do MAE, melhor o desempenho do modelo. Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:<br>

In [ ]:
mae = mean_absolute_error(y_test_exp, y_pred_exp)
print(f"Mean Absolute Error: {mae}")

#### Raiz do Erro Quadrático Médio (RMSE - do inglês *Root Mean Squared Error*)

O RMSE (Root Mean Squared Error) é uma métrica de avaliação utilizada em modelos de regressão que, assim como o MSE (Mean Squared Error), mede a diferença entre os valores reais e os valores previstos. No RMSE, essa diferença é elevada ao quadrado, o que elimina os sinais negativos e dá mais peso a grandes erros. A diferença principal está no fato de que, ao final, é aplicada a raiz quadrada no valor resultante, tornando a métrica mais interpretável, pois os erros são trazidos de volta à mesma escala das variáveis originais. Portanto, quanto menor o RMSE, melhor o desempenho do modelo.

In [ ]:
rmse = mean_squared_error(y_test_exp, y_pred_exp)
print(f"Root Mean Squared Error: {sqrt(rmse)}")

### Coeficiente de Determinação (R²)

O Coeficiente de Determinação R² é uma métrica que representa o percentual da variância dos dados previstos, ou seja, o quão explicativo é o modelo em relação aos dados de acordo com o quão distante esses valores estão do valor central (médio). Uma vez que a fórmula do R² considera a subtração desta conta por 1, quando menor o percentual obtido, melhor é a explicação do modelo. Este, no entanto, não é suficiente para ter uma noção geral da performace do modelo, dependendo de outras métricas (como o MAE e o MSE). Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:

In [ ]:
r2 = r2_score(y_test_exp, y_pred_exp)
print(f"R² Score: {r2}")

#### Erro Médio Absoluto Percentual (MAPE - do inglês Mean Absolute Percentage Error)

O MAPE é uma métrica utilizada para avaliar a precisão de modelos de regressão, calculando a média dos erros percentuais absolutos entre os valores reais e os valores preditos. Quanto menor o valor do MAPE, melhor o desempenho do modelo. Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:

In [ ]:

mape = mean_absolute_percentage_error(y_test_exp, y_pred_exp)
print (f"MAPE: {mape}")

### Conclusões a partir das métricas avaliadas

O modelo de Random Forest está funcionando muito bem com valores que variam de milhares a milhões. O R2 Score de 98,33% indica que o modelo explica quase toda a variabilidade dos dados. Além disso, o modelo faz previsões com precisão mais que suficiente para a maioria das aplicações práticas, porque tanto o MSE quanto o MAE e o MAPE são baixos em relação à escala dos dados fornecidos. Esses níveis de erro mostram que o modelo está bem calibrado e dá resultados confiáveis, uma vez que estão adequados para valores de grande magnitude como Valor Líquido da Rede Gazeta.

Por fim, teremos a representações de duas figuras relevantes para a compreensão do desempenho do modelo e uma mais fácil visualização de seus resultados.

Primeiro, temos uma comparação de valores reais e as previsões do modelo.

A linha roxa representa as previsões do modelo, enquanto os pontos vermelhos representam os valores reais que temos na tabela.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test_exp, y_pred_exp, color='red', alpha=0.6)
plt.plot([y_test_exp.min(), y_test_exp.max()], [y_test_exp.min(), y_test_exp.max()], color='purple', lw=3)  # Linha de identidade
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.title('Comparação entre Valores Reais e Previsões - Modelo Random Forest Regressor')
plt.grid(True)
plt.show()

E por último mas não menos importante, temos a representação gráfica da árvore de decisões, que representa as decisões nas quais o modelo se baseou para diferenciar as possibilidades presentes na base de dados.

In [15]:
# # Visualizando a árvore
# plt.figure(figsize=(16, 12))  # Ajuste o tamanho da figura conforme necessário
# plot_tree(modelo.estimators_[0], filled=True, feature_names=X.columns, rounded=True)
# plt.show()

### Explicabilidade
A explicabilidade de um modelo preditivo refere-se à capacidade de compreender e interpretar como um modelo toma suas decisões ou faz previsões. Isso é particularmente importante em modelos complexos onde o processo de decisão não é facilmente visível para os seres humanos. Logo abaixo, vamos iniciar a explicabilidade do modelo para conferir como as previsões foram realizadas.

In [ ]:
# Crie um objeto explainer SHAP
explainer = shap.Explainer(modelo, X_train)

# Calcule os valores SHAP para as previsões
shap_values = explainer(X_test, check_additivity=False)




#### Summary Plot
Esse gráfico de resumo mostra a importância global das variáveis no modelo, bem como a distribuição dos valores SHAP (impacto no resultado do modelo) para cada variável. Ele mostra como cada variável impacta globalmente o modelo e sua distribuição nos dados.

In [ ]:
shap.summary_plot(shap_values, X_test)

Explicações sobre a análise do gráfico:
- Eixo Vertical: Representa as variáveis do seu modelo. Cada linha corresponde a uma variável diferente que foi usada para fazer as previsões.
- Eixo Horizontal (SHAP values): Os valores SHAP indicam a contribuição de cada variável para aumentar ou diminuir a previsão do modelo. Se o valor SHAP é positivo, significa que essa variável contribui para aumentar a previsão do modelo. Se for negativo, ela contribui para diminuir a previsão.

As cores indicam o valor da variável:
- Azul: Valores baixos da variável.
- Vermelho: Valores altos da variável.
Portanto, um ponto vermelho para uma variável à direita significa que valores altos dessa variável aumentam a previsão, enquanto um ponto azul à esquerda indica que valores baixos diminuem a previsão.

Cada ponto representa um dado no conjunto de teste, e a dispersão dos pontos ao longo da linha da variável mostra como o impacto daquela variável pode variar dependendo dos seus valores.

As variáveis no topo têm maior impacto no resultado do modelo. Neste caso, `Segmento_COMERCIO` e `Segmento_SERVICOS` parecem ser os fatores mais importantes.

#### Waterfall Plot
Este gráfico explica a previsão individual de um dado ponto de teste. Ele decompõe o valor predito em contribuições individuais de cada variável. O valor final predito é a soma da base do modelo com as contribuições de cada variável. Ou seja, ele explica como cada variável contribui para a previsão de um ponto específico, mostrando o impacto individual de cada uma na previsão final. Ele mostra como as diferentes variáveis contribuíram para aumentar ou diminuir o valor previsto para um exemplo específico.

In [ ]:

# Mostra como cada recurso impactou uma previsão específica.
shap.waterfall_plot(shap_values[0])

Explicações sobre a análise do gráfico:
- Eixo Horizontal (f(x)): Mostra o valor previsto pelo modelo para esse exemplo específico. Neste caso, o valor final previsto é 9.128, que está representado no topo direito.
- Valor Base (E[f(X)]): É o valor médio de saída do modelo sem levar em consideração os efeitos das variáveis. Neste gráfico, o valor base é 8.758, que está mostrado na parte inferior.

Análisando as barras coloridas e o impacto das variáveis:
- Barras vermelhas: Representam variáveis que aumentaram o valor previsto do modelo em relação ao valor base.
- Barras azuis: Representam variáveis que diminuíram o valor previsto do modelo.
Cada barra mostra o quanto uma variável específica contribuiu para mover o valor da previsão em direção ao valor final. A magnitude (tamanho) da barra indica o impacto da variável.

Variáveis e seus Valores: À esquerda de cada barra está o nome da variável e o seu valor no exemplo. Por exemplo:
- Segmento_SERVICOS = 1 aumentou a previsão em +1.62.
- Veiculo_LITORAL FM NORTE = 1 diminuiu a previsão em -1.25.


O valor base é 8.758, e a previsão final é 9.128.
As variáveis Segmento_SERVICOS, Origem_VT - CONTATO - VITÓRIA, e algumas outras contribuíram para aumentar o valor previsto.
Por outro lado, variáveis como Veiculo_LITORAL FM NORTE, Veiculo_INTERNET, e Segmento_COMERCIO ajudaram a reduzir o valor da previsão.